<a href="https://colab.research.google.com/github/freakezoide/codigos-de-coolab/blob/main/modelo_con_limpieza_y_variantes_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import files

# 1. Subir archivos
uploaded = files.upload()
archivos = [f for f in uploaded.keys() if f.endswith('.csv')]

if not archivos:
    raise ValueError("No se subieron archivos CSV")

# 2. Cargar y combinar datos
df_list = []
for archivo in archivos:
    try:
        df = pd.read_csv(archivo)
        df_list.append(df)
    except Exception as e:
        print(f"Error al leer {archivo}: {str(e)}")

if not df_list:
    raise ValueError("No se pudo cargar ningún DataFrame")

data = pd.concat(df_list, ignore_index=True)

# 3. Inspección inicial
print("\n🔍 Análisis inicial:")
print("Columnas disponibles:", data.columns.tolist())
print("Total de registros:", len(data))
print("\nMuestra de datos:")
display(data.head(3))

# 4. Limpieza básica
print("\n🧹 Limpieza de datos:")
# Eliminar columnas completamente vacías
data.dropna(axis=1, how='all', inplace=True)

# Manejo de valores nulos
for col in data.columns:
    if data[col].isnull().sum() > 0:
        if data[col].dtype == 'object':
            data[col].fillna('Desconocido', inplace=True)
            print(f" - Nulos en {col} (categórica): rellenados con 'Desconocido'")
        else:
            median_val = data[col].median()
            data[col].fillna(median_val, inplace=True)
            print(f" - Nulos en {col} (numérica): rellenados con mediana {median_val:.2f}")

# 5. Ingeniería de características (ejemplo)
if all(col in data.columns for col in ['UsuariosCREA', 'UsuariosMAT', 'UsuariosBiblioteca']):
    data['Interaccion_Total'] = data['UsuariosCREA'] + data['UsuariosMAT'] + data['UsuariosBiblioteca']
    print("\n➕ Nueva característica creada: 'Interaccion_Total'")

# 6. Preprocesamiento
# Identificar automáticamente tipos de columnas
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = data.select_dtypes(include=['object', 'category']).columns.tolist()

# Eliminar la variable objetivo si existe
target = 'Interaccion_Total' if 'Interaccion_Total' in data.columns else None
if target:
    numeric_features.remove(target)

print("\n🔧 Columnas para transformación:")
print(" - Numéricas:", numeric_features)
print(" - Categóricas:", categorical_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 7. Modelado (ejemplo con regresión)
if target:
    X = data.drop(columns=[target])
    y = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    print("\n🤖 Entrenando modelo...")
    model.fit(X_train, y_train)

    # Evaluación
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("\n📊 Resultados:")
    print(f" - Error Cuadrático Medio (MSE): {mse:.2f}")
    print(f" - Coeficiente R²: {r2:.2f}")

    # Importancia de características
    if hasattr(model.named_steps['regressor'], 'feature_importances_'):
        feature_importances = model.named_steps['regressor'].feature_importances_
        # Obtener nombres de características después del preprocesamiento
        cat_encoder = model.named_steps['preprocessor'].named_transformers_['cat']
        cat_features = cat_encoder.get_feature_names_out(categorical_features)
        all_features = numeric_features + list(cat_features)

        importance_df = pd.DataFrame({
            'Característica': all_features,
            'Importancia': feature_importances
        }).sort_values('Importancia', ascending=False)

        print("\n🔝 Características más importantes:")
        display(importance_df.head(10))
else:
    print("\n⚠️ No se encontró variable objetivo para modelado")

Saving 2021 primaria - copia.csv to 2021 primaria - copia.csv
Saving 2022 primaria.csv to 2022 primaria.csv
Saving 2023 primaria.csv to 2023 primaria.csv
Saving 2019PRIMARIA (1).csv to 2019PRIMARIA (1).csv
Saving 2024 primaria.csv to 2024 primaria.csv
Saving 2020Primaria.csv to 2020Primaria.csv

🔍 Análisis inicial:
Columnas disponibles: ['Año', 'Mes', 'Departamento', 'Ciclo', 'Zona', 'Contexto', 'UsuariosCREA', 'UsuariosMAT', 'UsuariosBiblioteca']
Total de registros: 31654

Muestra de datos:


,Año,Mes,Departamento,Ciclo,Zona,Contexto,UsuariosCREA,UsuariosMAT,UsuariosBiblioteca
0,2021,Noviembre,Paysandu,Inicial,Urbana,Quintil 1 y 2,39,0,0
1,2021,Noviembre,Artigas,Inicial,Urbana,Quintil 3,1,0,0
2,2021,Noviembre,San Jose,Primaria 1er. Ciclo,Urbana,Quintil 1 y 2,447,213,19



🧹 Limpieza de datos:
 - Nulos en Contexto (categórica): rellenados con 'Desconocido'

➕ Nueva característica creada: 'Interaccion_Total'

🔧 Columnas para transformación:
 - Numéricas: ['Año', 'UsuariosCREA', 'UsuariosMAT', 'UsuariosBiblioteca']
 - Categóricas: ['Mes', 'Departamento', 'Ciclo', 'Zona', 'Contexto']

🤖 Entrenando modelo...


<ipython-input-1-91d2cd50e0f9>:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna('Desconocido', inplace=True)



📊 Resultados:
 - Error Cuadrático Medio (MSE): 1325.66
 - Coeficiente R²: 1.00

🔝 Características más importantes:


,Característica,Importancia
1,UsuariosCREA,0.986445
2,UsuariosMAT,0.010846
3,UsuariosBiblioteca,0.002226
0,Año,0.000107
43,Contexto_Quintil 1 y 2,0.000051
6,Mes_Agosto,0.000035
11,Mes_Mayo,0.000028
4,Mes_Abril,0.000026
14,Mes_Setiembre,0.000023
45,Contexto_Quintil 4 y 5,0.000023
